<a href="https://colab.research.google.com/github/elsa9421/Interactive-IPython-Demos/blob/main/1D_Gaussian_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


This notebook demonstrates a 1-D gaussian process regression example with several tunable parameters corresponding to Prior, Covariance function (by varying kernel parameters) and  noise

## Import

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
%matplotlib inline
from ipywidgets import interact,interactive,interactive_output, fixed, interact_manual, Label
import ipywidgets as widgets

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, RBF
import math


## Generate Data Samples

Let $y=f(x)+$$\epsilon$ , where $\epsilon$ ~ $N(0,$$\sigma_n$)
Here, $f(x)$ need not be a linear function of x.
>Consider the following 1-D problems: -
>>1. $f(x)=x$
>>2. $f(x)=sin(4\pi x)+sin(7\pi x)$
>>3. $f(x)=x sin(2\pi x)$


In [ ]:
plt.rcParams['figure.figsize'] = [12, 7]

def f1(x):
    f = x
    return(f)

def f2(x):
    f = np.sin((4*np.pi)*x) + np.sin((7*np.pi)*x)
    return(f)


def f3(x):
    f = x*np.sin(2*np.pi*x)
    return(f)
 





## Functions




### Kernel Function
Recall that a gaussian process is completely specified by its mean function and covariance (we usually take the mean equal to zero, although it is not necessary). 

Here, we will use the squared exponential kernel, also known as Gaussian kernel or RBF kernel:

$$
\kappa(\mathbf{x}_i,\mathbf{x}_j) = \sigma_f^2 \exp(-\frac{1}{2l^2}
  (\mathbf{x}_i - \mathbf{x}_j)^T
  (\mathbf{x}_i - \mathbf{x}_j))
$$

The length parameter $l$ controls the smoothness of the function and $\sigma_f$ the vertical variation. For simplicity, we use the same length parameter $l$ for all input dimensions (isotropic kernel). 



### Joint Distribution

The joint distribution of $y$ and $f_*$ is given by
\begin{equation}
\left( \begin{array}{ccc}
y\\
f_*\end{array} \right) = N(0,C)
\end{equation}

\begin{equation}
C= \left( \begin{array}{ccc}
\kappa(X,X)+\sigma_n^2 I & \kappa(X,X_*)\\
\kappa(X_*,X) & \kappa(X_*,X_*)\\
\end{array} \right)
\end{equation}
we need to add the term $\sigma_n^2 I$ to the upper left component to account for noise (assuming additive independent identically distributed Gaussian noise)


###  Statistics required for posterior predictive distribution

>>> $\mu_*$=$\kappa(X,X_*)^T$ $(\kappa(X,X)+\sigma_n^2 I)^{-1} y$

>>> $\Sigma_*$=$\kappa(X_*,X_*)-$$\kappa(X,X_*)^T$ $(\kappa(X,X)+\sigma_n^2 I)^{-1}\kappa(X,X_*)$


In [ ]:
def kernel2(X1, X2, l=1.0, sigma_f=1.0):
    '''
    Isotropic squared exponential kernel. Computes 
    a covariance matrix from points in X1 and X2.
    
    Args:
        X1: Array of m points (m x d).
        X2: Array of n points (n x d).

    Returns:
        Covariance matrix (m x n).
    '''
    sqdist = np.sum(X1**2, 1).reshape(-1, 1) + np.sum(X2**2, 1) - 2 * np.dot(X1, X2.T)
    return sigma_f**2 * np.exp(-0.5 / l**2 * sqdist)

def posterior_predictive(X_s, X_train, Y_train, l=1.0, sigma_f=1.0, sigma_n=1e-8):
    '''
    Computes the suffifient statistics of the GP posterior predictive distribution 
    from m training data X_train and Y_train and n new inputs X_s.
    
    Args:
        X_s: New input locations (n x d).
        X_train: Training locations (m x d).
        Y_train: Training targets (m x 1).
        l: Kernel length parameter.
        sigma_f: Kernel vertical variation parameter.
        sigma_n: Noise parameter.
    
    Returns:
        Posterior mean vector (n x d) and covariance matrix (n x n).
    '''
    kernel=kernel2
    K = kernel(X_train, X_train, l, sigma_f) + sigma_n**2 * np.eye(len(X_train))
    K_s = kernel(X_train, X_s, l, sigma_f)
    K_ss = kernel(X_s, X_s, l, sigma_f) + 1e-8  * np.eye(len(X_s))
    K_inv = np.linalg.inv(K)
    
    # Equation (4)
    mu_s = K_s.T.dot(K_inv).dot(Y_train)

    # Equation (5)
    cov_s = K_ss - K_s.T.dot(K_inv).dot(K_s)
    
    return K,K_s,K_ss,mu_s, cov_s

def plot_gp(mu, cov, X, X_train=None, Y_train=None, samples=[]):
    X = X.ravel()
    mu = mu.ravel()
    uncertainty = 1.96 * np.sqrt(np.diag(cov))
    
    plt.fill_between(X, mu + uncertainty, mu - uncertainty,color='k', alpha=0.1,label='95% confidence Interval')
    plt.plot(X, mu,'k', lw=2, zorder=9, label='GP Mean')
    for i, sample in enumerate(samples):
        plt.plot(X, sample, lw=1, ls='--', label=f'Sample {i+1}')
    if X_train is not None:
        plt.scatter(X_train, Y_train, c='b', s=20, zorder=10, edgecolors=(0, 0, 0), label='f($X_{train}+\epsilon$)')
    plt.legend(loc='lower right')

## Plot portfolio

In [ ]:
def plot_portfolio(sigma_epsilon=0,sigma_n=0,f=2,sigma_f=2,l=0.1):
  '''
  The function plots GP mean?

  Input :
  sigma_epsilon=Error standard deviation introduced in data
  sigma_n:  noise parameter introduced in Cov matrix
  f      : option to select function object to be applied on training data X_train
  X_train: 1-D training data
  sigma_f: Kernel vertical variation parameter. 
  l: Kernel length parameter.
 
  '''
  sns.set()

  plt.rcParams['figure.figsize'] = [24, 8]
  if f==1:
    f=f1
  if f==2:
    f=f2
  if f==3:
    f=f3
  #plt.figure(figsize=(16,6))

  # Converting l:
  l=round(math.exp(l),3)

  # Train Data

  n = 20
  x_train = np.linspace(start=0, stop=1, num=n)
  # Errors.
  np.random.seed(1)
  epsilon = np.random.normal(loc=0, scale=sigma_epsilon, size=n)
  # Actual target variable
  f_x=f(x_train)
   # Observed target variable. (with noise)
  y_train = f_x + epsilon
  

  #Test Data
  n_star = 100
  x_star = np.linspace(start=0, stop=1, num=n_star) 


  # Plot 1 : The True data, and train data points

  # plt.figure()
  # # Plot training data.
  # plt.scatter(x_train, y_train, c='b', s=20, zorder=10, edgecolors=(0, 0, 0), label='f($x_{train}+N(0,\sigma_{epsilon}^2)$)');
  # # Plot "true" linear fit.
  # sns.lineplot(x=x_star, y=f(x_star), color='red', label='f(x)');
  # plt.title('Sample Data, $\sigma_{epsilon}$={}'.format(sigma_epsilon))
  # plt.xlabel("x")
  # plt.ylabel("f(x)")
  # plt.legend(loc='lower right')

  #-----------------------------------------------------------------------------------
  
  # # Plot Prior
  np.random.seed(1)
  epsilon_star = np.random.normal(loc=0, scale=sigma_epsilon, size=n_star)
  y_star=f(x_star)+epsilon_star


  plt.subplot(1,2,1)
  kernel =  ConstantKernel(sigma_f)* RBF(length_scale=l)
  gp = GaussianProcessRegressor(kernel=kernel,alpha=sigma_n**2, n_restarts_optimizer=10)
  #gp = GaussianProcessRegressor(kernel=kernel,alpha=sigma_n**2)

  y_mean, y_std = gp.predict(x_star[:, np.newaxis], return_std=True)
  plt.plot(x_star, y_mean, 'k', lw=3, zorder=9,label='$Prior_{mean}$')
  plt.plot(x_star, f(x_star), 'r-', label='f(x)');
  plt.fill_between(x_star, y_mean - y_std, y_mean + y_std,
                     alpha=0.2, color='k')
  y_samples = gp.sample_y(x_star[:, np.newaxis], 3)
  plt.plot(x_star, y_samples[:,0],color= 'orange',linestyle='dashed',lw=1,label='Sample 1')
  plt.plot(x_star, y_samples[:,1], color='blue',linestyle='dashed',lw=1,label='Sample 2')
  plt.plot(x_star, y_samples[:,2], color='green',linestyle='dashed',lw=1,label='Sample 3')
  plt.xlim(0, 1)
  plt.title("Prior (kernel:  %s)" % kernel, fontsize=12)
  plt.xlabel("x")
  plt.ylabel("f(x)")
  plt.legend(loc="lower right")

  #-------------------------------------------------------------------------------------


 

  plt.subplot(1,2,2)

  _,_,_,mu_s, cov_s = posterior_predictive(x_star.reshape(-1,1), x_train.reshape(-1,1), y_train.reshape(-1,1), l=l, 
                                       sigma_f=sigma_f, 
                                       sigma_n=sigma_n)

  plt.title("GP Mean=$\kappa(X,X_*)^T (\kappa(X,X)+\sigma_n^2 I)^{-1} y$"+" where l = {}, sigma_f = {}, sigma_n = {} ".format(l,sigma_f,sigma_n))
  plt.plot(x_star, f(x_star), 'r', label='f(x)')
  plt.xlabel("x")
  plt.ylabel("f(x)")
  plot_gp(mu_s, cov_s, x_star, x_train[:, np.newaxis], Y_train=y_train)






  #-------------------------------------------------------------------------------------
#  Fit to data  - To  find out optimized hyperparameters
  # plt.figure()
  # X_train=x_train[:, np.newaxis]
  # # The hyperparameters of the kernel are optimized during fitting of GaussianProcessRegressor by maximizing the log-marginal-likelihood (LML) 
  # gp.fit(X_train, y_train[:, np.newaxis])

  # # Plot posterior
  # plt.subplot(1,2,1)
  # y_mean, y_std = gp.predict(x_star[:, np.newaxis], return_std=True)
  # #y_mean, y_std = gp.predict(x_star[:, np.newaxis], return_cov=True)
  # plt.plot(x_star, y_mean, 'k', lw=2, zorder=9,label='GP Mean')

  # # print(x_star.shape)
  # y_mean=y_mean.reshape(-1)
  # # print(y_std.shape)
  # plt.fill_between(x_star, y_mean - y_std, y_mean + y_std,
  #                   alpha=0.2, color='k',label='GP Mean+/-std_deviation')

  # # y_samples = gp.sample_y(X_test[:, np.newaxis], 3)
  # # plt.plot(X_test, y_samples, lw=1)
  # plt.scatter(X_train[:, 0], y_train, c='b', s=20, zorder=10, edgecolors=(0, 0, 0), label='f($X_{train}+\epsilon$)')
  # plt.plot(x_star, f(x_star), 'r', label='f(x)');
  # plt.xlim(0, 1)
  # plt.title("Maximizing the log marginal likelihood:  Posterior (kernel: %s)\n Log-Likelihood: %.3f"
  #           % (gp.kernel_, gp.log_marginal_likelihood(gp.kernel_.theta)),
  #           fontsize=12)
  # plt.xlabel("x")
  # plt.ylabel("f(x)")
  # plt.legend(loc="lower right")


#-------------------------------------------------------------------------




Function_widget=widgets.Dropdown(
                      options=[('x', 1), ('sin(4\u03C0x)+sin(7\u03C0x)', 2), ('xsin(2\u03C0ix)', 3)],
                      value=1,
                      description='f(x)=')

sigma_f_slider=widgets.FloatSlider( value=0.703,
                                 min=0.1,
                                 max=1.5,
                                 step=.1,
                                 continuous_update=False)



l_slider=widgets.FloatSlider( value=math.log(.93),
                                 min=-7,                                  # min=math.log(.01),
                                 max=8,                                   # max=math.log(3),
                                 step=math.log(.01),                      # step of 1
                                 description=' ln(0.93)=  :',
                                 continuous_update=False)


def update_sigm_f_l(*args):
  '''
  To update sigma_f and l to the optimal parameters 
  using maximum marginal likelihood based on given function input

  '''
  if Function_widget.value==1:
    sigma_f_slider.value= 0.703
    l_slider.value=math.log(0.93)
  elif Function_widget.value==2:
    sigma_f_slider.value=0.877
    l_slider.value=math.log(0.0458)
  elif Function_widget.value==3:
    sigma_f_slider.value=0.416
    l_slider.value=math.log(0.173)
  l_slider.description=' ln({})=  :'.format(round(math.exp(l_slider.value),3))


def update_description(*args):
  '''
  To update l_slider description
  '''
  l_slider.description=' ln({})=  :'.format(round(math.exp(l_slider.value),3))

Function_widget.observe(update_sigm_f_l,'value')
l_slider.observe(update_description,'value')


sigma_n_slider=widgets.FloatSlider(value=0.3,
                                 min=0.01,
                                 max=3,
                                 step=.1,
                                 continuous_update=False)

sigma_epsilon_slider=widgets.FloatSlider(value=0.3,
                                 min=0.01,
                                 max=3,
                                 step=.1,
                                 continuous_update=False)

#Noise_widget=widgets.HBox([Label('Noise Parameter (Covariance Matrix)   :'), sigma_n_slider])


kernel_widget=widgets.HBox([Label('Kernel Parameters (Covariance Matrix)    :'), Label('Vertical Variation    :'),
                            sigma_f_slider,Label('Length Variation  :' ),l_slider,Label('Noise Parameter:'), sigma_n_slider])

Function_Noise_widget=widgets.HBox([Function_widget,Label('Noise introduced in data   :'),
                            sigma_epsilon_slider])


out=interactive_output(plot_portfolio,{"sigma_epsilon":sigma_epsilon_slider,"sigma_n":sigma_n_slider,"sigma_f":sigma_f_slider,"l":l_slider,"f":Function_widget})
display(Function_Noise_widget,kernel_widget,out)



Output()